# Heartbeat Detection

## Imports

In [2]:
import copy
import wave
from pathlib import Path
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, Conv1D, AvgPool1D, MaxPool1D, ZeroPadding1D, BatchNormalization, Flatten, Dense, Activation, Dropout
from keras.utils.data_utils import get_file
from keras.utils.np_utils import to_categorical
import math

## Load heartbeat sounds

In [33]:
# Classes to handle, ordered by label
CLASSES = ['artifact','arrythmia','normal', 'noise']
DATASET_DIR = Path('dataset_heartbeat')

x_train = []
y_train = []
x_test = []
y_test = []

for recording in DATASET_DIR.glob(f'**/*.wav'):
    if not recording.parent.name in CLASSES: # Ignore unused classes
        continue
    label = CLASSES.index(recording.parent.name) # Assign class number
    
    with wave.open(str(recording)) as f: # Read wave file
        data = np.frombuffer(f.readframes(f.getnframes()), dtype=np.int16).copy() # As 16-bit signed integer
        
    data = data.astype(np.float32) # Convert to 32-bit floating-point
    data.resize((16000, 1)) # Resize to 1s (16kHz) with zero-padding, 1 channel
    
    x_train.append(data)
    y_train.append(label)

x_train = np.array(x_train)
y_train = to_categorical(np.array(y_train))

perms2 = np.random.permutation(len(y_train))[0:math.ceil(len(y_train)*0.2)]

x_test = x_train[perms2]
y_test = y_train[perms2]

x_train = np.delete(x_train, perms2, axis=0)
y_train = np.delete(y_train, perms2, axis=0)

perms1 = np.random.permutation(len(y_train))[0:math.ceil(len(y_train)*0.1)]

x_val = x_train[perms1]
y_val = y_train[perms1]

x_train = np.delete(x_train, perms1, axis=0)
y_train = np.delete(y_train, perms1, axis=0)

print(x_train.shape)
print(x_val.shape)
print(x_test.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(661, 16000, 1)
(74, 16000, 1)
(184, 16000, 1)
(661, 4)
(74, 4)
(184, 4)


## Prepare for inference with fixed-point Q7.9 samples by scaling input data accordingly

In [34]:
FIXED_POINT = 9
x_train /= 2**FIXED_POINT
x_test  /= 2**FIXED_POINT

## Build model *not* M5

In [40]:
# Define hyperparameters
num_classes = len(CLASSES) # Number of sound classes
num_filters = 2 # Number of filters for CNN layers
num_units = 128 # Number of units for RNN layers
num_heads = 8 # Number of heads for attention layers
dropout_rate = 0.1 # Dropout rate for regularization

model = Sequential()
model.add(Input(shape=(16000, 1)))
model.add(MaxPool1D(pool_size=4))
model.add(Conv1D(2, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=4))
model.add(Conv1D(4, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=4))
model.add(Conv1D(8, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=4))
model.add(Conv1D(16, kernel_size=3, activation='relu'))
model.add(Conv1D(32, kernel_size=3, activation='relu'))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=4))
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(6, activation='relu'))
model.add(Dense(units=num_classes, activation='softmax')) # SoftMax activation needs to be separate from Dense to remove it later on
# EXPLORE Learning Rate
opt = tf.keras.optimizers.Adam(learning_rate=10e-6)
model.summary()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 max_pooling1d_25 (MaxPoolin  (None, 4000, 1)          0         
 g1D)                                                            
                                                                 
 conv1d_30 (Conv1D)          (None, 3998, 2)           8         
                                                                 
 max_pooling1d_26 (MaxPoolin  (None, 999, 2)           0         
 g1D)                                                            
                                                                 
 conv1d_31 (Conv1D)          (None, 997, 4)            28        
                                                                 
 max_pooling1d_27 (MaxPoolin  (None, 249, 4)           0         
 g1D)                                                            
                                                      

## Train model

In [41]:
model.fit(x_train, y_train, epochs=70, batch_size=10, validation_data=(x_val, y_val))

Epoch 1/70
67/67 [==============================] - 2s 17ms/step - loss: 1.6615 - categorical_accuracy: 0.2390 - val_loss: 372.8742 - val_categorical_accuracy: 0.3243
Epoch 2/70
67/67 [==============================] - 1s 14ms/step - loss: 1.4788 - categorical_accuracy: 0.3162 - val_loss: 319.2780 - val_categorical_accuracy: 0.3919
Epoch 3/70
67/67 [==============================] - 1s 14ms/step - loss: 1.3531 - categorical_accuracy: 0.3858 - val_loss: 279.4325 - val_categorical_accuracy: 0.4459
Epoch 4/70
67/67 [==============================] - 1s 14ms/step - loss: 1.2523 - categorical_accuracy: 0.4206 - val_loss: 246.0269 - val_categorical_accuracy: 0.4865
Epoch 5/70
67/67 [==============================] - 1s 14ms/step - loss: 1.1748 - categorical_accuracy: 0.4539 - val_loss: 219.6659 - val_categorical_accuracy: 0.4865
Epoch 6/70
67/67 [==============================] - 1s 14ms/step - loss: 1.1189 - categorical_accuracy: 0.5038 - val_loss: 199.8509 - val_categorical_accuracy: 0.513

Epoch 50/70
67/67 [==============================] - 1s 15ms/step - loss: 0.7140 - categorical_accuracy: 0.7337 - val_loss: 113.1399 - val_categorical_accuracy: 0.6757
Epoch 51/70
67/67 [==============================] - 1s 14ms/step - loss: 0.7083 - categorical_accuracy: 0.7383 - val_loss: 109.4961 - val_categorical_accuracy: 0.6892
Epoch 52/70
67/67 [==============================] - 1s 14ms/step - loss: 0.7037 - categorical_accuracy: 0.7398 - val_loss: 108.3655 - val_categorical_accuracy: 0.7027
Epoch 53/70
67/67 [==============================] - 1s 14ms/step - loss: 0.6981 - categorical_accuracy: 0.7413 - val_loss: 106.6073 - val_categorical_accuracy: 0.7027
Epoch 54/70
67/67 [==============================] - 1s 16ms/step - loss: 0.6939 - categorical_accuracy: 0.7428 - val_loss: 102.4634 - val_categorical_accuracy: 0.7162
Epoch 55/70
67/67 [==============================] - 1s 15ms/step - loss: 0.6886 - categorical_accuracy: 0.7489 - val_loss: 102.1750 - val_categorical_accuracy:

## Evaluate model on test dataset

In [43]:
model.evaluate(x_test, y_test, verbose=2)
pred_test = model.predict(x_test)
print(tf.math.confusion_matrix(y_test.argmax(axis=1), pred_test.argmax(axis=1)))

6/6 - 0s - loss: 0.6997 - categorical_accuracy: 0.7391 - 80ms/epoch - 13ms/step
6/6 [==============================] - 0s 13ms/step
tf.Tensor(
[[ 0  4  7  0]
 [ 0 10 24  0]
 [ 0  8 68  0]
 [ 0  0  5 58]], shape=(4, 4), dtype=int32)


## Save trained model

In [44]:
model.save('heartbeat_74percent.h5')

np.savetxt('x_test.csv', x_test.reshape((x_test.shape[0], -1)), delimiter=',', fmt='%s')
np.savetxt('y_test.csv', y_test, delimiter=',', fmt='%s')

## Remove SoftMax layer

In [45]:
model = tf.keras.Model(model.input, model.layers[-2].output, name=model.name)

## Install MicroAI for C inference code generation (kerascnn2c module)

In [46]:
%pip install https://bitbucket.org/edge-team-leat/microai_public/get/6adfbcb347d3.zip#subdirectory=third_party/kerascnn2c_fixed
import kerascnn2c

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: HTTPSConnectionPool(host='bitbucket.org', port=443): Max retries exceeded with url: /edge-team-leat/microai_public/get/6adfbcb347d3.zip (Caused by NewConnectionError('<pip._vendor.urllib3.connection.HTTPSConnection object at 0x0000022B401E2AA0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))


[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Generate C code for the trained model with 16-bit fixed-point representation

In [47]:
res = kerascnn2c.Converter(output_path=Path('gsc_output_fixed'),
                           fixed_point=FIXED_POINT, # Number of bits for the fractional part, Q7.9 format
                           number_type='int16_t', # Data type for weights/activations (16 bits quantization)
                           long_number_type='int32_t', # Data type for intermediate results
                           number_min=-(2**15), # Minimum value for 16-bit signed integers
                           number_max=(2**15)-1 # Maximum value for 16-bit signed integers
                          ).convert_model(copy.deepcopy(model))
with open('full_model.h', 'w') as f:
    f.write(res)

———————————————————————————————————————————————————————————————————————————————————————————————————————
Inputs                           | Layer                            | Outputs                         
———————————————————————————————————————————————————————————————————————————————————————————————————————
                                 | input_6                          | max_pooling1d_25                
-------------------------------------------------------------------------------------------------------
input_6                          | max_pooling1d_25                 | conv1d_30                       
-------------------------------------------------------------------------------------------------------
max_pooling1d_25                 | conv1d_30                        | max_pooling1d_26                
-------------------------------------------------------------------------------------------------------
conv1d_30                        | max_pooling1d_26                 

## Compile the 16-bit fixed-point C code for x86 and evaluate on small dataset

In [48]:
!g++ -Wall -Wextra -pedantic -Ofast -o gsc_fixed -Igsc_output_fixed/ gsc_output_fixed/model.c main.cpp 
!./gsc_fixed x_test.csv y_test.csv

gsc_output_fixed/model.c: In function 'void cnn(const number_t (*)[16000], number_t*)':
gsc_output_fixed/model.c:178:18: warning: left operand of comma operator has no effect [-Wunused-value]
  178 |     activations1.max_pooling1d_29_output,
      |     ~~~~~~~~~~~~~^~~~~~~~~~~~~~~~~~~~~~~
'.' is not recognized as an internal or external command,
operable program or batch file.
